In [16]:
import more_itertools as mit

data_path = r'C:/Users/10inm/NLP/princess_bride.txt'

# Defining lines as a list of each line
with open(data_path, 'r', encoding='utf-8') as f:
  raw_lines = f.read().split('\n')

raw_lines.reverse()
lines = []

for line in raw_lines:
    # split line into parts
    line_split = line.split(' +++$+++ ')
    # append tuple of character and line
    line_num = int(line_split[0][1:])

    current_line = line_split[4].strip()
    # append tuple of line num, character and line
    lines.append((line_num, current_line))
# make sure the lines are in order
lines = sorted(lines, key=lambda x: x[0])

# group lines by scene
by_scene = [list(group) for group in mit.consecutive_groups(lines, lambda x: x[0])]

dialog_only = [[dialog_line[1] for dialog_line in dialog_group] 
                for dialog_group in by_scene]

dialog_combos_nested = [list(map(list, zip(dialog_group, dialog_group[1:]))) for dialog_group in dialog_only]

dialog_combos = [combo for combos in dialog_combos_nested for combo in combos]

# print dialog combos:
print(dialog_combos)


ValueError: invalid literal for int() with base 10: ''

In [18]:
dialog_combos = [['You feeling any better?', 'A little bit.'],
                 ['A little bit.', 'Guess what.'], ['Guess what.', 'What?'],
                 ['What?', "Your grandfather's here."],
                 ["Your grandfather's here.", "Mom, can't you tell him that I'm sick?"],
                 ["Mom, can't you tell him that I'm sick?", "You are sick, that's why he's here."],
                 ['I brought you a special present.', 'What is it?'], ['What is it?', 'Open it up.'],
                 ['Open it up.', 'A book?'], ['A book?', "That's right."],
                 ["He's climbing the rope. And he's gaining on us.", 'Inconceivable!'],
                 ['Sorry.', 'Thank you.'], ['Thank you.', 'I do not suppose you could speed things up?'],
                 ['Hello.', "It's you."], ["It's you.", 'True!'],
                 ['True!', "You don't look so good.  You don't smell so good either."],
                 ["You don't look so good.  You don't smell so good either.", 'Perhaps not. I feel fine.'],
                 ['Perhaps not. I feel fine.', 'Yeah?'], ['An hour.', 'Yeah, an hour.'],
                 ['Yeah, an hour.', 'A good hour. Yeah.'],
                 ['Now I think you ought to go to sleep.', 'Okay.'], ['Okay.', 'Okay. Okay. Okay. All right. So long.'],
                 ['Maybe you could come over and read it again to me tomorrow.', 'As you wish...']]

In [20]:
import numpy as np
from seq2seq import encoder_model, decoder_model, num_decoder_tokens, target_features_dict, reverse_target_features_dict, max_decoder_seq_length

class ChatBot:
  
  negative_responses = ("no", "nope", "nah", "naw", "not a chance", "sorry")

  exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop")
  
  def start_chat(self):
    user_response = input("Hi, I'm a chatbot trained on dialog from The Princess Bride. Would you like to chat with me?\n")
    
    if user_response in self.negative_responses:
      print("Ok, have a great day!")
      return
    
    self.chat(user_response)
  
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply))
    

  def generate_response(self, user_input):
    states_value = encoder_model.predict(user_input)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_features_dict['<START>']] = 1.

    chatbot_response = ''

    stop_condition = False
    while not stop_condition:
      output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)

      sampled_token_index = np.argmax(output_tokens[0, -1, :])
      sampled_token = reverse_target_features_dict[sampled_token_index]
      chatbot_response += " " + sampled_token

      if (sampled_token == '<END>' or len(chatbot_response) > max_decoder_seq_length):
        stop_condition = True

      target_seq = np.zeros((1, 1, num_decoder_tokens))
      target_seq[0, 0, sampled_token_index] = 1.

      states_value = [hidden_state, cell_state]

    return chatbot_response
  
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Ok, have a great day!")
        return True
      
    return False
  
chatty_mcchatface = ChatBot()
# call .generate_response():
chatty_mcchatface.generate_response("asdasdas")



Cool!

